## Check cconv Implementation

In [3]:
def enable_parent_folder_import():
    import sys, os
    sys.path.insert(1, os.path.join(sys.path[0], '..'))

enable_parent_folder_import()

In [2]:
from importlib import reload
import datasets.density_data_module
from datasets.density_data_module import *
reload(datasets.density_data_module); from datasets.density_data_module import *

density_data = DensityDataModule(
    data_dir = '../datasets/data/dpi_dam_break/train',
    batch_size = 10,
    data_split = (0.7, 0.15, 0.15),
    num_workers = 0, # Note that cuda only allows 0 workers.
    shuffle = False,
    cache = True, # Load dataset into memory
    device = 'cuda',
)
density_data.setup("fit")
train_loader = density_data.train_dataloader()
train_iter = iter(train_loader)
batch = next(train_iter)
sample = batch[0]

Setting up data module for stage  fit
Loading dataset into memory and applying transform_once... (Device: cuda)


  0%|          | 0/131 [00:00<?, ?it/s]

100%|██████████| 131/131 [00:16<00:00,  7.84it/s]

Done loading dataset into memory


In [8]:
from importlib import reload

import models.cconv
from models.cconv import *
reload(models.cconv); from models.cconv import *

hparams = {}
hparams['batch_size'] = 10
hparams['lr'] = 1e-3

model = CConvModel(hparams)
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# model(sample)

CConvModel(
  (param_list): ParameterList(
      (0): Object of type: ContinuousConv
      (1): Object of type: ContinuousConv
      (2): Object of type: Linear
      (3): Object of type: Linear
      (4): Object of type: ContinuousConv
      (5): Object of type: Linear
      (6): Object of type: ContinuousConv
      (7): Object of type: Linear
      (8): Object of type: ContinuousConv
    (0): ContinuousConv(
      (fixed_radius_search): FixedRadiusSearch()
      (radius_search): RadiusSearch()
    )
    (1): ContinuousConv(
      (fixed_radius_search): FixedRadiusSearch()
      (radius_search): RadiusSearch()
    )
    (2): Linear(in_features=5, out_features=32, bias=True)
    (3): Linear(in_features=96, out_features=64, bias=True)
    (4): ContinuousConv(
      (fixed_radius_search): FixedRadiusSearch()
      (radius_search): RadiusSearch()
    )
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): ContinuousConv(
      (fixed_radius_search): FixedRadiusSearch()
    

In [118]:
model.configure_optimizers()

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [124]:
from utils.visualization import *
reload(utils.visualization); from utils.visualization import *

from utils.callbacks import *
reload(utils.callbacks); from utils.callbacks import *

dataset = density_data.dataset['train']
vis3d = VisulizePrediction3DCallback(model, dataset)

In [128]:
# plot_sample_for_tensorboard(model, dataset)

In [126]:
as_tensor = fig_to_tensor(plot_sample_for_tensorboard(model, dataset))

In [127]:
as_tensor.shape

torch.Size([4, 400, 1250])

In [ ]:
model = CConvModel(hparams)
trainer = pl.Trainer()
trainer.fit(model, datamodule=density_data, ckpt_path='../checkpoints/lightning_logs/version_0/checkpoints/epoch=29-step=870.ckpt')

In [4]:
from importlib import reload

import models.cconv
from models.cconv import *
reload(models.cconv); from models.cconv import *

p = '../lightning_logs/2023-06-30_17-01-19_checkpoint.ckpt'
model = CConvModel.load_from_checkpoint(p)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [17]:
checkpoint = torch.load(p)
for key in checkpoint.keys():
    print(key)

epoch
global_step
pytorch-lightning_version
state_dict
loops
callbacks
optimizer_states
lr_schedulers
hyper_parameters


In [18]:
model = CConvModel(hparams)
model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [5]:
# import pytorch lightning
import pytorch_lightning as pl

trainer = pl.Trainer()
print("Version of current run (trainer.logger.version): {}".format(trainer.logger.version))
    

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/jakob/Bachelorarbeit/GNNDensityGradients/notebooks/lightning_logs


Version of current run (trainer.logger.version): 0


In [ ]:
# load trainer and everything from checkpoint
trainer = pl.Trainer(resume_from_checkpoint=p)
print("Version of current run (trainer.logger.version): {}".format(trainer.logger.version))
